<a href="https://colab.research.google.com/github/rogerfvieira/beak_length_prediction/blob/main/beak_length_prediction(linear_regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade google-cloud-bigquery
# !pip install google-colab

# Fare prediction (Linear_regression) using bigqeury ML

## Imports


In [1]:
from google.cloud import bigquery 
from google.colab import auth
import pandas as pd

In [2]:
auth.authenticate_user()

## Cursor creation

In [6]:
project_id = 'play-368717'
dataset_id = 'linear_regression'
client = bigquery.Client(project=project_id)

## Unclean data source

In [7]:
unclean_data_query= """
SELECT *
FROM `bigquery-public-data.ml_datasets.penguins`
LIMIT 100
"""
df_unclean = client.query(unclean_data_query).to_dataframe()

In [8]:
df_unclean

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie Penguin (Pygoscelis adeliae),Dream,36.6,18.4,184.0,3475.0,FEMALE
1,Adelie Penguin (Pygoscelis adeliae),Dream,39.8,19.1,184.0,4650.0,MALE
2,Adelie Penguin (Pygoscelis adeliae),Dream,40.9,18.9,184.0,3900.0,MALE
3,Chinstrap penguin (Pygoscelis antarctica),Dream,46.5,17.9,192.0,3500.0,FEMALE
4,Adelie Penguin (Pygoscelis adeliae),Dream,37.3,16.8,192.0,3000.0,FEMALE
...,...,...,...,...,...,...,...
95,Adelie Penguin (Pygoscelis adeliae),Dream,36.5,18.0,182.0,3150.0,FEMALE
96,Adelie Penguin (Pygoscelis adeliae),Dream,41.1,19.0,182.0,3425.0,MALE
97,Adelie Penguin (Pygoscelis adeliae),Dream,36.0,17.9,190.0,3450.0,FEMALE
98,Adelie Penguin (Pygoscelis adeliae),Dream,41.1,17.5,190.0,3900.0,MALE


## Data Cleaning / feature engineering and selection

In [9]:
training_data_query = """
CREATE OR REPLACE TABLE `play-368717.linear_regression.flipper_length_prediction_training_data`
AS(SELECT species,
       island,
       culmen_length_mm,
       culmen_depth_mm,
       flipper_length_mm,
       body_mass_g,
       sex 
 FROM `bigquery-public-data.ml_datasets.penguins`
 WHERE species IS NOT NULL
 AND island IS NOT NULL
 AND culmen_depth_mm IS NOT NULL
 AND culmen_depth_mm IS NOT NULL
 AND flipper_length_mm IS NOT NULL
 AND body_mass_g IS NOT NULL
 AND sex IS NOT NULL);
  )
"""
training_data= client.query(training_data_query)

## Clean data

In [10]:
dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
table_ref = dataset_ref.table("flipper_length_prediction_training_data")
table = client.get_table(table_ref)
df_training_data = client.list_rows(table).to_dataframe()

In [11]:
df_training_data

,species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Gentoo penguin (Pygoscelis papua),44.5,15.7,217.0,4875.0,.
1,Gentoo penguin (Pygoscelis papua),47.6,14.5,215.0,5400.0,MALE
2,Gentoo penguin (Pygoscelis papua),49.1,15.0,228.0,5500.0,MALE
3,Gentoo penguin (Pygoscelis papua),49.6,15.0,216.0,4750.0,MALE
4,Gentoo penguin (Pygoscelis papua),50.7,15.0,223.0,5550.0,MALE
...,...,...,...,...,...,...
329,Adelie Penguin (Pygoscelis adeliae),37.3,17.8,191.0,3350.0,FEMALE
330,Adelie Penguin (Pygoscelis adeliae),36.6,17.8,185.0,3700.0,FEMALE
331,Adelie Penguin (Pygoscelis adeliae),39.5,17.8,188.0,3300.0,FEMALE
332,Adelie Penguin (Pygoscelis adeliae),36.7,18.8,187.0,3800.0,FEMALE


## Model Creation

In [12]:
model_creation_query = """
CREATE OR REPLACE MODEL `play-368717.linear_regression.flipper_length_prediction_model`
OPTIONS(MODEL_TYPE = 'LINEAR_REG',
    INPUT_LABEL_COLS = ['flipper_length_mm'],
    CATEGORY_ENCODING_METHOD ='DUMMY_ENCODING') AS SELECT * FROM `play-368717.linear_regression.`;
"""

In [13]:
fare_prediction_model = client.query(model_creation_query)

In [15]:
models = client.list_models('linear_regression')

In [18]:
for elements in models:
  print(elements)

Model(reference=ModelReference(project_id='play-368717', dataset_id='linear_regression', model_id='fare_prediction_model'))
Model(reference=ModelReference(project_id='play-368717', dataset_id='linear_regression', model_id='flipper_length_prediction_model'))


## Model Evaluation

In [19]:
model_evaluation_query = '''
SELECT * FROM ML.EVALUATE(MODEL `play-368717.linear_regression.flipper_length_prediction_model`)
'''

In [20]:
model_eval =  client.query(model_evaluation_query).to_dataframe()

In [21]:
model_eval

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,3.964646,26.734707,0.000686,3.116867,0.863621,0.863621
